# Testa om history h1-h5 kan användas ensamt


In [ ]:
!pip install bayesian-optimization
import pandas as pd
import numpy as np
import fixa_features as ff
import fixa_mer_features as ff2

from catboost import CatBoostClassifier as CB
from catboost import Pool,cv
from catboost.utils import get_confusion_matrix 
from bayes_opt import BayesianOptimization
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

pd.set_option('mode.chained_assignment',None)

df=pd.read_csv('mer_komplett.csv')
# df.drop('tid',axis=1,inplace=True)

# df=ff2.fixa_mer_features(df,hist=True)
df.lopp_dist=df.lopp_dist.astype('float')
df.plac = (df.plac==1)*1
print(df.plac.value_counts())  # CatBoost fungerar bättre med 0,1 än False, True
print(df.start.value_counts())

In [ ]:
df[df.delta1.isna()]

In [ ]:
df.info()

In [ ]:
#  df.datum=df.datum.astype('string')
 df.kmtid
 

In [ ]:
def hx_dat(dat):
    dat = dat.astype('string').str.replace('.0','',regex=False)
    dat = dat.str.replace('(','',regex=False)
    dat = dat.str.replace(')','',regex=False)
    dat=pd.to_datetime(dat)
    return dat

In [ ]:
df.datum=pd.to_datetime(df.datum)
df.h1_dat = hx_dat(df.h1_dat)
df.h2_dat = hx_dat(df.h2_dat)
df.h3_dat = hx_dat(df.h3_dat)
df.h4_dat = hx_dat(df.h4_dat)
df.h5_dat = hx_dat(df.h5_dat)

df[['datum','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat']].info()


In [ ]:
#### gör inte detta ####
# dfh.h1_plac=(dfh.h1_plac==1)*1
# dfh.h2_plac=(dfh.h2_plac==1)*1
# dfh.h3_plac=(dfh.h3_plac==1)*1
# dfh.h4_plac=(dfh.h4_plac==1)*1
# dfh.h5_plac=(dfh.h5_plac==1)*1

def ber_perf(pris,plac):
    return np.sqrt(pris * np.exp(15-plac))

# performance prissumma*plac. Ju högre desto bättre
# dfh['h1_perf']=(dfh.h1_pris * (20-dfh.h1_plac)).describe()
# dfh['h2_perf']=(dfh.h2_pris * (20-dfh.h2_plac)).describe()
# dfh['h3_perf']=(dfh.h3_pris * (20-dfh.h3_plac)).describe()
# dfh['h4_perf']=(dfh.h4_pris * (20-dfh.h4_plac)).describe()
# dfh['h5_perf']=(dfh.h5_pris * (20-dfh.h5_plac)).describe()

df['h1_perf'] = ber_perf(df.h1_pris,df.h1_plac)
df['h2_perf'] = ber_perf(df.h2_pris,df.h2_plac)
df['h3_perf'] = ber_perf(df.h3_pris,df.h3_plac)
df['h4_perf'] = ber_perf(df.h4_pris,df.h4_plac)
df['h5_perf'] = ber_perf(df.h5_pris,df.h5_plac)



In [ ]:
# ny kolumn för delta mellan aktuellt opp och h1_dat
df['senast'] = df.datum-df.h1_dat

df.info()

In [ ]:
# nya kolumner för datumavstånd
df['delta1'] = df.h1_dat-df.h2_dat
df['delta2'] = df.h2_dat-df.h3_dat
df['delta3'] = df.h3_dat-df.h4_dat
df['delta4'] = df.h4_dat-df.h5_dat
df[['senast','delta1','delta2','delta3','delta4']].describe()



In [ ]:
df.to_csv('mer_komplett.csv',index=False,date_format='%s')

In [ ]:
# selektera ut history-datat
dfh=df[['häst','kusk','bana','datum','h1_bana', 'h1_dat', 'h1_dist','h1_kmtid', 'h1_kusk', 'h1_odds', 'h1_plac', 'h1_pris', 'h1_spår','h2_bana', 'h2_dat', 'h2_dist', 'h2_kmtid', 'h2_kusk', 'h2_odds', 'h2_plac', 'h2_pris', 'h2_spår', 'h3_bana', 'h3_dat', 'h3_dist', 'h3_kmtid', 'h3_kusk', 'h3_odds', 'h3_plac', 'h3_pris', 'h3_spår', 'h4_bana', 'h4_dat', 'h4_dist', 'h4_kmtid', 'h4_kusk', 'h4_odds','h4_plac', 'h4_pris', 'h4_spår', 'h5_bana', 'h5_dat', 'h5_dist', 'h5_kmtid', 'h5_kusk', 'h5_odds', 'h5_plac', 'h5_pris', 'h5_spår','plac']]

In [ ]:
# selektera ut kolumner för train och test
dfh.columns
selected_features=['häst','kusk','bana', 'senast', 'delta1','delta2', 'delta3', 'delta4', 
'h1_perf','h1_bana', 'h1_dist', 'h1_kmtid', 'h1_kusk','h1_odds', 'h1_plac', 'h1_pris', 'h1_spår', 
'h2_perf','h2_bana', 'h2_dist', 'h2_kmtid', 'h2_kusk','h2_odds', 'h2_plac', 'h2_pris', 'h2_spår', 
'h3_perf','h3_bana', 'h3_dist', 'h3_kmtid', 'h3_kusk','h3_odds', 'h3_plac', 'h3_pris', 'h3_spår', 
'h4_perf','h4_bana', 'h4_dist', 'h4_kmtid', 'h4_kusk','h4_odds', 'h4_plac', 'h4_pris', 'h4_spår', 
'h5_perf','h5_bana', 'h5_dist', 'h5_kmtid', 'h5_kusk','h5_odds', 'h5_plac', 'h5_pris', 'h5_spår']

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(dfh[selected_features],dfh.plac,test_size=0.3,random_state=2021)


In [ ]:
def catfit(trainPool, testPool,auto=None,eval='AUC',use_best=False,early=200,iterations=2000):
      
    model = CB( 
        random_seed=2021,
        iterations=iterations,   
        bagging_temperature = 10.968228520463521, 
        depth = 4, 
        l2_leaf_reg= 2, 
        learning_rate = 0.02980857933628237,         
        auto_class_weights='Balanced' ,
        # scale_pos_weight = 10,
        eval_metric = 'Accuracy',
        loss_function = 'Logloss',
        early_stopping_rounds=early,
        verbose = 100
        )
    if use_best:
        model.fit(trainPool,
        use_best_model=True,
        eval_set=testPool,
        verbose = 100
        )
    else:            
        model.fit(trainPool,
        use_best_model=False,
        eval_set=testPool,
        verbose = 100
        )
    
    return model



In [ ]:
 
from catboost import Pool
from catboost.utils import get_confusion_matrix 
def training(trainPool,testPool):
    use_best = True
    early=300
    metrics='Accuracy'
    iterations=2000
    model = catfit(trainPool, testPool, auto='Balanced',eval=metrics,use_best=use_best,early=early,iterations=iterations)
    return model

In [ ]:
add_features=['senast', 'delta1','delta2', 'delta3', 'delta4',
'h1_perf', 'h1_dist', 'h1_kmtid', 'h1_odds',   
'h2_perf', 'h2_dist', 'h2_kmtid', 'h2_odds',
'h3_perf', 'h3_dist', 'h3_kmtid', 'h3_odds', 
'h4_perf', 'h4_dist', 'h4_kmtid', 'h4_odds', 
'h5_perf', 'h5_dist', 'h5_kmtid', 'h5_odds']
cat_features = ['häst','kusk','bana','h1_bana', 'h1_kusk', 'h2_bana', 'h2_kusk','h3_bana', 'h3_kusk','h4_bana', 'h4_kusk','h5_bana', 'h5_kusk']
sel_features = cat_features+add_features
trainPool = Pool(X_train[sel_features],y_train,cat_features=cat_features)
testPool = Pool(X_test[sel_features],y_test,cat_features=cat_features)

model = training(trainPool,testPool)

print(get_confusion_matrix(model,testPool))
eval=model.eval_metrics(testPool,['Accuracy','F1','Precision','Recall','AUC'])

print(f"Acc={round(np.mean(eval['Accuracy']),3)} F1={round(np.mean(eval['F1']),3)} Recall={round(np.mean(eval['Recall']),3)} Precision={round(np.mean(eval['Precision']),3)} AUC={round(np.mean(eval['AUC']),3)}")

In [ ]:
fi=model.get_feature_importance(prettified=True)
fi

In [ ]:
# from catboost import CatBoostClassifier as CB

model = CB(  
        random_seed=2021,
        iterations=iterations,   
        bagging_temperature = 10.968228520463521, 
        depth = 4, 
        l2_leaf_reg= 2, 
        learning_rate = 0.02980857933628237,         
        auto_class_weights='Balanced' ,
        # scale_pos_weight = 10,
        eval_metric = 'Accuracy',
        loss_function = 'Logloss',
        early_stopping_rounds=early,
        
        # use_best_model=True,
        
        verbose = 100
        )
    
# grid = {
#         'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}

# grid_search_result = model.grid_search(grid,
#                                        trainPool,  
                                    
#                                       cv=5)


# Bayesian Optimization

In [ ]:
def bayes():
        pds = { 'learning_rate':(0.005,0.07),
           'depth': (1,12),
          'l2_leaf_reg': (1,9),
          'bagging_temperature': (7,12)
          }
        optimizer = BayesianOptimization(hyp_cb1, pds,random_state=2021)
                                        
        # Optimize (init_points: how many initial random searches to do)
        optimizer.maximize(init_points=2, n_iter=18)
        print(optimizer.max)
        return optimizer

{'target': 0.6505510125907216, 'params': {'bagging_temperature': 10.39169546060608, 'depth': 3.666313874920603, 'l2_leaf_reg': 4.474218266848133, 'learning_rate': 0.01539416204116328}}  

{'target': 0.6681093548342557, 'params': {'bagging_temperature': 10.968228520463521, 'depth': 4.123578340791053, 'l2_leaf_reg': 2.365031119669437, 'learning_rate': 0.02980857933628237}}


In [ ]:

def hyp_cb1(learning_rate, depth, l2_leaf_reg, bagging_temperature):

    params = {'iterations': 1500,
              'loss_function': 'Logloss',
              'eval_metric': 'Accuracy',
              'auto_class_weights': 'Balanced' ,
              'early_stopping_rounds': 200, 
              'use_best_model': True,
              # 'eval_set': testPool,  #behövs inte för cv
              'random_seed': 2021,
              "verbose": False
    }

    params["learning_rate"] = learning_rate
    params[ "depth"] = int(round(depth)) 
    params["l2_leaf_reg"] = int(round(l2_leaf_reg))
    params["bagging_temperature"] = bagging_temperature
    
    cat_features = ['häst','kusk','bana','h1_bana', 'h1_kusk', 'h2_bana', 'h2_kusk','h3_bana', 'h3_kusk','h4_bana', 'h4_kusk','h5_bana', 'h5_kusk']
    
    trainPool = Pool(data=dfh[sel_features],
                    label=dfh.plac,
                    cat_features=cat_features)

    scores = cv(trainPool,
                   params,
                   fold_count=3)

    return np.max(scores['test-Accuracy-mean'])  



In [ ]:
optimizer = bayes()
optimizer


In [ ]:
# spara modell
model.save_model('modeller_history/model_senaste',
        format="cbm",
        export_parameters=None,
        pool=None)